# 다양한 분류 모형

In [1]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler

from sklearn.naive_bayes import MultinomialNB
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn import neighbors
from sklearn.svm import SVC

import re
import numpy as np
import pandas as pd


- logistic regression

In [ ]:
from sklearn.datasets import load_wine
dataset = load_wine() 

dataset.keys()
dataset['data']
dataset['target'] 
dataset['DESCR']
dataset['feature_names']

Y=dataset['target']>1


In [ ]:
import numpy as np
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
features = scaler.fit_transform(dataset['data'])
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(features, Y
, test_size=0.3)


from sklearn import linear_model
logistic = linear_model.LogisticRegression()
logistic.fit(X_train, Y_train)
print(logistic.coef_)
print(np.exp(logistic.coef_))
print(logistic.score(X_test, Y_test)) #정분류율
print(logistic.predict(X_test[1:3,]))	#테스트셋 첫 두 행에 대한 예측
print(logistic.predict_proba(X_test[1:3,]))

- k-NN과 SVM

In [ ]:
#creditset.csv를 읽으세요
csv_data = pd.read_csv("creditset.csv")
csv_data.shape
csv_data.isnull().mean()

clientid       0.0
income         0.0
age            0.0
loan           0.0
LTI            0.0
default10yr    0.0
dtype: float64

In [ ]:
X = csv_data[ ["age", "income", "loan"]]
y = csv_data.default10yr

#Partitioning
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42 )
scaler = preprocessing.StandardScaler()
X_train = scaler.fit_transform( X_train)
X_test  = scaler.fit_transform( X_test  )

In [ ]:
#kNN
knn = neighbors.KNeighborsClassifier()
#n_neighbors, default=5
#metric: default "minkowski", default이고 p=2로 지정하면 유클리드 거리
#n_jobs: numer of parallel jobs, -1이면 컴퓨터 CPU 코어 수로 지정됨

knn.fit(X_train, y_train)
knn.kneighbors(X_test, return_distance=False)
knn.predict_proba(X_test)
predicted = knn.predict(X_test)

knn.score(X_test, y_test)
np.mean( predicted == y_test )




#n_neighbors를 11과 21, 31, 41로 바꾸셔서 정분류율을 비교해보세요.



0.9783333333333334

In [ ]:
# binary.csv를 읽으셔서 knn 적용하세요 ->sp500_tm.csv
binary = pd.read_csv("sp500_tm.csv")
scaled = preprocessing.StandardScaler().fit_transform( binary.drop( "Direction", 
                                                                   axis=1))
X_train, X_test, y_train, y_test = train_test_split( scaled, binary.Direction, 
                                                    test_size=0.2)

In [9]:
for i in [5,7,9,11,13]:
  knn = neighbors.KNeighborsClassifier(n_neighbors=i).fit( X_train, y_train)
  print( i, knn.score(X_test, y_test) ) #accuracy 출력
from sklearn.metrics import classification_report
#print( classification_report(  knn.predict( X_test), y_test ) )

5 0.35
7 0.3
9 0.25
11 0.3
13 0.3


In [ ]:
# creditset.csv를 읽으셔서 1) MinMaxScaling 2) 8:2 partitioning
# Y: default10yr, X: loan, age, income
# model 1) dt, 2)rf, 3) knn, 성능은 accuracy로 비교
# rf의 경우 n_estimators를 50, 100, 150로 변화하면서 성능을 비교하세요
# knn의 경우, n_neighbors를 5,7,9,11로 변화하면서 성능을 비교하세요

In [11]:
credit = pd.read_csv( "creditset.csv", index_col=0 )
credit.drop("LTI", axis=1, inplace=True)
scaled = preprocessing.MinMaxScaler().fit_transform( 
    credit[["loan","income","age"]]   )
X_train, X_test, y_train, y_test = train_test_split( scaled, 
                                                    credit.default10yr,
                                                    test_size=0.2)

In [13]:
dt = tree.DecisionTreeClassifier().fit( X_train, y_train)
print( "dt", dt.score( X_test, y_test ))
for i in [50,100,150]:
  rf = RandomForestClassifier(n_estimators=i).fit(X_train, y_train)
  print( "rf", i, rf.score( X_test, y_test))
for j in [5,7,9,11]:
  knn = neighbors.KNeighborsClassifier(n_neighbors=j).fit(X_train, y_train)
  print( "knn", j, knn.score( X_test, y_test))

rf = RandomForestClassifier(n_estimators=50).fit(X_train, y_train)
print( classification_report(  rf.predict(X_test), y_test) )

dt 0.9825
rf 50 0.9925
rf 100 0.99
rf 150 0.99
knn 5 0.9825
knn 7 0.98
knn 9 0.98
knn 11 0.975
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       340
           1       0.97      0.97      0.97        60

    accuracy                           0.99       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.99      0.99      0.99       400



In [15]:
#SVM
clf = SVC().fit(X_train, y_train)
#C : penalty, default 1.0
#kernel: linear, poly, rbf, sigmoid, precomputed, default: rbf
#degree: polynmial degree, default 3
#gamma: flaot, default auto
#probability: boolean, default=FALSE, predict_proba를 사용하게 함
#tol: float, default=1e-3
predicted = clf.predict(X_test)
clf.score(X_test, y_test)

0.9725

- 위의 작업을 DT, RF로도 수행해보세요.(scaling 적용)


In [ ]:
dt1 = tree.DecisionTreeClassifier().fit(X_train, y_train)
pred = dt1.predict( X_test)
np.mean( pred == y_test )

rf = RandomForestClassifier(n_estimators=100, oob_score=True, random_state=123456).fit(X_train, y_train)
predicted = rf.predict(X_test)
np.mean( predicted == y_test )



0.985

- 다음 CSV 파일에 대해 DT, RF, kNN, SVM의 성능을 비교해보세요

In [ ]:
#sp500_tm.csv
#4개 변수 중 1~3변수가 X 변수, 4번째가 Y변수
#7:3으로 파티셔닝하세요
#X변수는 표준화하세요